In [19]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
from astropy.table import Table, vstack, Column
from astropy.io import ascii
import scipy, pylab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits as pyfits

In [138]:
# ----Cut out objects clustered around bright stars using starcoords ---- #
def nthneighbor(ra, dec):
    # input ra, dec arrays -- we can call each objects as c[i]
    c = SkyCoord(ra, dec, unit='deg')
    
    idx, seps, _ = c.match_to_catalog_sky(c,nthneighbor=3)
    minsep = 60 # arcsecs
    cut = seps.arcsec > minsep   # make sure we're using proper units, want to look at objects 
                                 # in separation threshold to see if we've cleaned up the data
    return cut                   # boolean array of indexes we want to use

In [21]:
def starmask(ra, dec):
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    
    #take out bright stars as well
    # legacy candidates
    #cat = Table.read('legacy-lsbg-candidates.fits')

    # Tycho-2 catlaog from Pickles & Depagne (2010) 
    tb = Table.read('tycho.fits')
    tb.rename_column('RAJ2000', 'ra')
    tb.rename_column('DEJ2000', 'dec')

    # mag cut
    tb = tb[tb['Vfmag']<11.5]

    # position cut
    pos_cut = (((tb['ra'] > 290) | (tb['ra'] < 80)) & (tb['dec'] > -21))
    pos_cut |= (((tb['ra'] < 260) & (tb['ra'] > 120)) & (tb['dec'] > -10))
    tb = tb[pos_cut]
    
    min_sep = 60 # arcsec

    cat_coord = SkyCoord(ra, dec, unit='deg')
    tycho_coord = SkyCoord(tb['ra'], tb['dec'], unit='deg')
    idx, sepz, _ = cat_coord.match_to_catalog_sky(tycho_coord)
    #print((sepz.arcsec > min_sep).sum(), 'candidates near bright stars')

    cut = sepz.arcsec > min_sep    # get all objects that are above this minimum separation from bright stars
    
    return cut

In [161]:
# create function to extract files from each tractor file

def extract_tractor(directory):

    #directory = "/scratch/jgreco/tractor/191" # the directory we're going to work with
    # initialize all the arrays I want to construct
    
    columns = ['flux_r', 'flux_g', 'flux_z', 'shapeexp_e1', 'shapeexp_e2', 'psfsize_r']
    apfluxes = ['apflux_r', 'apflux_g', 'apflux_z']
    
    flux_r = []
    flux_g = []
    flux_z = []
    shapeexp_r = []
    shapeexp_e1 = []
    shapeexp_e2 = []
    apflux_r = [] 
    apflux_g = []
    apflux_z = []
    
    csb_g = []
    
    psfsize_r = []
    
    apid = ['0.5', '0.75', '1.0', '1.5', '2.0', '3.5', '5.0', '7.0']
    aparray_r = [[] for i in range(8)]
    aparray_g = [[] for i in range(8)] # create empty list of lists to fill with all aperture vals
    aparray_z = [[] for i in range(8)] 

    # coordinate info
    objid = []
    typ = []
    ra = []
    dec = []


    for fname in os.listdir(directory):     # iterate through the directory
        filename = os.fsdecode(fname)

        if filename.endswith(".fits"):
            #print(os.path.join(directory, filename))

            file = os.path.join(directory, filename)
            t2 = pyfits.open(file)         # open file using pyfits
            data = t2[1].data              # get the data from the file
             
            
                
            t2.close()                     # close file so we don't get bogged down
            
            
            # construct boolean mask to enforce our cuts (and so that we don't overload the for loop)
            keep = (data['flux_r']>0.0) & (data['flux_r']<1e3)
            keep &= (data['type']!="PSF ")
            
            #cols = data.columns.names[16:]  
            #names = cols.names[3:]   # want everything except ra, dec data (clean out gross numbers)
            names = [cols for column in cols if column.startswith('apflux')]
            for i in range(len(names)):
                keep &= (data[names[i]]>0.0) & (data[names[i]]<1e3)
                keep &= ~np.isnan(data[names[i]])
                keep &= ~np.isinf(data[names[i]])
            
            #for ap in range(len(apfluxes)):
                #for i in range(8):
                    #keep &= data[apfluxes[ap]][:,i] > 0.0
                    #keep &= data[apfluxes[ap]][:,i] < 1e3
                    #keep &= ~np.isnan(data[apfluxes[ap]][:,i])
                    #keep &= ~np.isinf(data[apfluxes[ap]][:,i])
                
            # keep the cuts by overwriting the data table
            
            t = data[keep]
            
            
            
            # convert model fluxes to magnitudes and make cuts
            magg = (22.5-2.5*np.log10(t["flux_g"]))
            magr = (22.5-2.5*np.log10(t["flux_r"]))
            magz = (22.5-2.5*np.log10(t["flux_z"]))

            mask = (magg < 22.5)
            mask &= (magg>19.)
            mask &= (magr<23.) & (magz<23.)
            mask &= (magr>19.) & (magz>19.)
            
            #t = t[mask]
            
            
            # cut out objects with central surface brightness (2.0 arcsec ap flux) brighter than 19.
            ap_r = 22.5 - 2.5*np.log10(t["apflux_r"][:,4])
            ap_g = 22.5 - 2.5*np.log10(t["apflux_g"][:,4])
            ap_z = 22.5 - 2.5*np.log10(t["apflux_z"][:,4])
                
            mask &= (ap_g < 22.5)                
            mask &= (ap_r<23.) & (ap_z<23.)
            mask &= (ap_g > 19.)  
            mask &= (ap_r > 19.)
            mask &= (ap_z > 19.)
            
            
            # get size information and make an exponential radius cut at 5 arcseconds
            radius = 5.0    #arcseconds
            mask &= t['shapeexp_r'] > radius
            # update table
            #t = t[mask1]
            
            
            # make nth neighbor and bright star cuts
            # first take out stars
           # if len(t['ra'] > 1):
               # mask &= starmask(t['ra'], t['dec'])
               # mask &= nthneighbor(t['ra'], t['dec'])
            
            

            
            # for central surface brightness
            aperture = 2.0  # central aperture
            csb_g = ap_g + 2.5*np.log10((np.pi*((aperture**2))))           # central surface brightness of aperture in g band
            
            
            # mask over the final table
            
            tfinal = t[mask].copy()

            # get coordinate info
            typ = np.concatenate((np.asarray(typ), tfinal["type"]), axis=0)
            objid = np.concatenate((np.asarray(objid), tfinal["objid"]), axis=0)
            ra = np.concatenate((np.asarray(ra), tfinal["ra"]), axis=0)
            dec = np.concatenate((np.asarray(dec), tfinal["dec"]), axis=0)

            #psf_size for r band
            psfsize_r = np.concatenate((np.asarray(psfsize_r), tfinal["psfsize_r"]), axis=0)

            # for r band data
            flux_r = np.concatenate((np.asarray(flux_r), magr[mask]), axis=0)  # assemble the big flux_r array in mags
            shapeexp_r = np.concatenate((np.asarray(shapeexp_r), tfinal["shapeexp_r"]), axis=0)
            shapeexp_e1 = np.concatenate((np.asarray(shapeexp_e1), tfinal["shapeexp_e1"]), axis=0)
            shapeexp_e2 = np.concatenate((np.asarray(shapeexp_e2), tfinal["shapeexp_e2"]), axis=0)



            # for g band data
            flux_g = np.concatenate((np.asarray(flux_g), magg[mask]), axis=0)  # assemble the big flux_r array
            
            csb_g = np.concatenate((np.asarray(flux_g), csb_g[mask]), axis=0)
            # for z band data 
            flux_z = np.concatenate((np.asarray(flux_z), magz[mask]), axis=0)  # assemble the big flux_r array 
            
            
            
            
            for ap in range(len(apid)):   # fill in all aperture arrays in list of arrays
                # convert aperture fluxes to magnitudes
                
                rmag = 22.5 - 2.5*np.log10(tfinal["apflux_r"][:,ap])
                gmag = 22.5 - 2.5*np.log10(tfinal["apflux_g"][:,ap])
                zmag = 22.5 - 2.5*np.log10(tfinal["apflux_z"][:,ap])
                
                aparray_r[ap] = np.concatenate((np.asarray(aparray_r[ap]), rmag)) # get all aperture mags
                aparray_g[ap] = np.concatenate((np.asarray(aparray_g[ap]), gmag))
                aparray_z[ap] = np.concatenate((np.asarray(aparray_z[ap]), zmag))
                
    # once we're done concatenating all the arrays we want, let's put the results into a table
    
    t = Table()
    
    t['ra'] = Column(ra, unit='deg', description='object right ascension')
    t['dec'] = Column(dec, unit='deg', description='object declination')

    t['type'] = Column(typ, unit = '', description='object type char[4]')
    t['objid'] = Column(objid, unit='', description='integer ID tag for object')

    t['flux_r'] = Column(flux_r, unit='magnitude', description='flux_r')
    t['flux_g'] = Column(flux_g, unit='magnitude', description='flux_g')
    t['flux_z'] = Column(flux_z, unit='magnitude', description='flux_z')

    # aperture fluxes
    for i in range(8):
        apname_r = "apflux_" + apid[i] + "_r"
        apname_g = "apflux_" + apid[i] + "_g"
        apname_z = "apflux_" + apid[i] + "_z"
        t[apname_r] = Column(aparray_r[i], unit='magnitude', description = "apname_r")
        t[apname_g] = Column(aparray_g[i], unit='magnitude', description = "apname_g")
        t[apname_z] = Column(aparray_z[i], unit='magnitude', description = "apname_z")

    t['shapeexp_r'] = Column(shapeexp_r, unit='arcsec', description = 'exponential shape half-light radius')
    t['shapeexp_e1'] = Column(shapeexp_e1, unit='arcsec', description = 'ellipticipy argument 1')
    t['shapeexp_e2'] = Column(shapeexp_e2, unit='arcsec', description = 'ellipticipy argument 2')
    t['psfsize_r'] = Column(psfsize_r, unit='arcsec', description = 'weighted average PSF FWHM in r band')
    t['csb_g'] = Column(csb_g, unit='mag/arcsec^2', description = 'central surface brightness in g-band')
    
    # return the table object
    return t


In [157]:
directory = "/scratch/jgreco/tractor/191"

test = extract_tractor(directory)

/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in log10
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in less
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in greater
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in log10
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in log10
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in less
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in greater
/u/tmakinen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value en

In [158]:
test

ra,dec,type,objid,flux_r,flux_g,flux_z,apflux_0.5_r,apflux_0.5_g,apflux_0.5_z,apflux_0.75_r,apflux_0.75_g,apflux_0.75_z,apflux_1.0_r,apflux_1.0_g,apflux_1.0_z,apflux_1.5_r,apflux_1.5_g,apflux_1.5_z,apflux_2.0_r,apflux_2.0_g,apflux_2.0_z,apflux_3.5_r,apflux_3.5_g,apflux_3.5_z,apflux_5.0_r,apflux_5.0_g,apflux_5.0_z,apflux_7.0_r,apflux_7.0_g,apflux_7.0_z,shapeexp_r,shapeexp_e1,shapeexp_e2,psfsize_r,csb_g
deg,deg,,,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,arcsec,arcsec,arcsec,arcsec,mag / arcsec2
float64,float64,str32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
191.110298532,9.85795382269,EXP,3839.0,20.5157585144,21.3045520782,19.7919445038,23.7890472412,24.8690128326,22.9819049835,22.8733730316,24.0472564697,22.1257324219,22.2684326172,23.3741836548,21.5007591248,21.3624153137,22.4030761719,20.5841331482,20.6528053284,21.7020721436,19.8988761902,18.9300746918,20.0135307312,18.1795578003,17.7377319336,18.966758728,16.9988651276,17.4305591583,18.6318874359,16.6931648254,9.00383472443,0.387841403484,0.574083387852,1.22929275036,21.3045520782
191.059790781,-2.83581317278,EXP,574.0,22.3042316437,21.2378177643,21.5871257782,25.0610141754,24.4170742035,24.0995159149,24.2463378906,23.5336074829,23.353761673,23.6653575897,22.9592819214,22.8644599915,22.7583389282,22.1969623566,22.1282787323,22.2551383972,21.6296482086,21.5605831146,21.3447723389,20.4404678345,20.6310901642,20.6547737122,19.6790294647,19.8717288971,19.8815670013,18.9168815613,19.1330814362,9.06830120087,0.21750818193,0.0399470999837,1.53298985958,21.2378177643
191.232040483,-4.0317101102,EXP,1665.0,22.5905227661,20.8571929932,20.3772754669,24.7964954376,23.9630661011,23.5303478241,23.8392353058,23.420425415,22.432641983,23.1750888824,22.9750423431,21.7866630554,22.1607437134,22.0472011566,20.7616329193,21.1221866608,21.0537014008,19.9563732147,18.6310348511,19.1854667664,18.3685379028,18.5570354462,18.9702606201,18.2089195251,18.5426368713,18.932138443,18.2079582214,8.38926029205,0.963504314423,0.00737554905936,1.0675868988,20.8571929932
191.226003596,-4.52583348018,EXP,1881.0,20.365032196,21.0554885864,19.7771434784,24.2883548737,24.8388233185,23.888010025,23.4851989746,23.9168663025,22.9818458557,22.7958869934,23.2721805573,22.440196991,21.8706455231,22.4043292999,21.4932498932,21.2254981995,21.7707862854,20.8504886627,20.0078353882,20.4943466187,19.5994720459,19.0588264465,19.49741745,18.6587715149,17.9999275208,18.4472389221,17.6568813324,6.29788064957,0.260893493891,0.403354197741,1.04239475727,21.0554885864
191.11050581,0.462994976749,EXP,1499.0,21.0184421539,21.4213924408,20.642665863,24.3101844788,24.7169361115,24.215297699,23.5540008545,23.8646793365,23.3362827301,22.9271850586,23.2640266418,22.6522350311,21.9956741333,22.338727951,21.7318134308,21.3057136536,21.7456035614,21.0989532471,20.1026077271,20.5157642365,19.8157444,19.2800102234,19.6708698273,19.0168304443,18.5371398926,18.9204654694,18.2914772034,6.95670366287,0.881433963776,0.0832177847624,1.39708721638,21.4213924408
191.206811863,1.24448776847,EXP,2356.0,20.3829536438,20.6897258759,20.0479049683,24.3868427277,24.8181686401,23.7710876465,23.3771419525,23.9178352356,22.807559967,22.7263717651,23.2298069,22.1985092163,21.9636440277,22.4076023102,21.3947181702,21.3503456116,21.774936676,20.8224334717,19.7748565674,20.2598323822,19.3614692688,18.8324165344,19.3011741638,18.4212970734,17.5199337006,18.1305789948,17.0108299255,5.82090854645,0.198906138539,0.573101580143,1.23810589314,20.6897258759
191.203485749,1.24646129687,EXP,2357.0,19.6301383972,19.8907012939,19.3

In [139]:
cut = starmask(test['ra'], test['dec'])
cut &= nthneighbor(test['ra'], test['dec'])

71

In [140]:
test2 = test[cut].copy() 

In [145]:
test2

ra,dec,type,objid,flux_r,flux_g,flux_z,apflux_0.5_r,apflux_0.5_g,apflux_0.5_z,apflux_0.75_r,apflux_0.75_g,apflux_0.75_z,apflux_1.0_r,apflux_1.0_g,apflux_1.0_z,apflux_1.5_r,apflux_1.5_g,apflux_1.5_z,apflux_2.0_r,apflux_2.0_g,apflux_2.0_z,apflux_3.5_r,apflux_3.5_g,apflux_3.5_z,apflux_5.0_r,apflux_5.0_g,apflux_5.0_z,apflux_7.0_r,apflux_7.0_g,apflux_7.0_z,shapeexp_r,shapeexp_e1,shapeexp_e2,psfsize_r,csb_g
deg,deg,,,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,magnitude,arcsec,arcsec,arcsec,arcsec,mag / arcsec2
float64,float64,str32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
191.110298532,9.85795382269,EXP,3839.0,20.5157585144,21.3045520782,19.7919445038,23.7890472412,24.8690128326,22.9819049835,22.8733730316,24.0472564697,22.1257324219,22.2684326172,23.3741836548,21.5007591248,21.3624153137,22.4030761719,20.5841331482,20.6528053284,21.7020721436,19.8988761902,18.9300746918,20.0135307312,18.1795578003,17.7377319336,18.966758728,16.9988651276,17.4305591583,18.6318874359,16.6931648254,9.00383472443,0.387841403484,0.574083387852,1.22929275036,21.3045520782
191.232040483,-4.0317101102,EXP,1665.0,22.5905227661,20.8571929932,20.3772754669,24.7964954376,23.9630661011,23.5303478241,23.8392353058,23.420425415,22.432641983,23.1750888824,22.9750423431,21.7866630554,22.1607437134,22.0472011566,20.7616329193,21.1221866608,21.0537014008,19.9563732147,18.6310348511,19.1854667664,18.3685379028,18.5570354462,18.9702606201,18.2089195251,18.5426368713,18.932138443,18.2079582214,8.38926029205,0.963504314423,0.00737554905936,1.0675868988,20.8571929932
191.226003596,-4.52583348018,EXP,1881.0,20.365032196,21.0554885864,19.7771434784,24.2883548737,24.8388233185,23.888010025,23.4851989746,23.9168663025,22.9818458557,22.7958869934,23.2721805573,22.440196991,21.8706455231,22.4043292999,21.4932498932,21.2254981995,21.7707862854,20.8504886627,20.0078353882,20.4943466187,19.5994720459,19.0588264465,19.49741745,18.6587715149,17.9999275208,18.4472389221,17.6568813324,6.29788064957,0.260893493891,0.403354197741,1.04239475727,21.0554885864
191.11050581,0.462994976749,EXP,1499.0,21.0184421539,21.4213924408,20.642665863,24.3101844788,24.7169361115,24.215297699,23.5540008545,23.8646793365,23.3362827301,22.9271850586,23.2640266418,22.6522350311,21.9956741333,22.338727951,21.7318134308,21.3057136536,21.7456035614,21.0989532471,20.1026077271,20.5157642365,19.8157444,19.2800102234,19.6708698273,19.0168304443,18.5371398926,18.9204654694,18.2914772034,6.95670366287,0.881433963776,0.0832177847624,1.39708721638,21.4213924408
191.206811863,1.24448776847,EXP,2356.0,20.3829536438,20.6897258759,20.0479049683,24.3868427277,24.8181686401,23.7710876465,23.3771419525,23.9178352356,22.807559967,22.7263717651,23.2298069,22.1985092163,21.9636440277,22.4076023102,21.3947181702,21.3503456116,21.774936676,20.8224334717,19.7748565674,20.2598323822,19.3614692688,18.8324165344,19.3011741638,18.4212970734,17.5199337006,18.1305789948,17.0108299255,5.82090854645,0.198906138539,0.573101580143,1.23810589314,20.6897258759
191.203485749,1.24646129687,EXP,2357.0,19.6301383972,19.8907012939,19.3483104706,24.2625102997,24.7657680511,23.8406467438,23.3922710419,23.826467514,22.9735355377,22.7977466583,23.20728302,22.3428840637,21.8692817688,22.2891559601,21.4553222656,21.2133846283,21.6737976074,20.8039951324,19.9690513611,20.4339580536,19.5676422119,18.954908371,19.4327659607,18.5353336334,17.494562149,18.1221847534,16.9762382507,7.82062292099,0.399626463652,0.41414642334,1.23962640762,19.8907012939
191.24198633,2.13799408603,COMP,469.0,21.0288829803,21.8183383942,20.3787822

In [148]:
output = test2.copy()

output.rename_column('objid', 'id')
output.rename_column('csb_g', 'csb')
output['id', 'ra', 'dec', 'shapeexp_r', 'csb' ].write('tractor-func-test.fits', overwrite=True)

In [81]:
ids = Table.read('best-legacy-lsbgs.csv')

In [120]:
for i in range(len(t['objid'])):
    for j in range(len(ids['candidate_objid'])):
        if t['objid'][i] == ids['candidate_objid'][j]:
            print(t['objid'][i], t['ra'][i], t['dec'][i])

749.0 191.652399121 -4.07374760412
329.0 191.849916532 0.642942758837


In [88]:
cut

False

In [ ]:
# Now let's put all of the arrays into a table object
# ***Note: in Astropy.Table, it's much easier to add columns side-by-side than it is to vstack rows
t = Table()

#t['objid'] = Column(objid, unit='int32', description='Legacy objid')
t['ra'] = Column(ra, unit='deg', description='object right ascension')
t['dec'] = Column(dec, unit='deg', description='object declination')

t['type'] = Column(typ, unit = '', description='object type char[4]')
t['objid'] = Column(objid, unit='', description='integer ID tag for object')

t['flux_r'] = Column(flux_r, unit='nanomaggies', description='flux_r')
t['flux_g'] = Column(flux_g, unit='nanomaggies', description='flux_g')
t['flux_z'] = Column(flux_z, unit='nanomaggies', description='flux_z')

# aperture fluxes

for i in range(8):
    apname_r = "apflux_" + apid[i] + "_r"
    apname_g = "apflux_" + apid[i] + "_g"
    apname_z = "apflux_" + apid[i] + "_z"
    t[apname_r] = Column(aparray_r[i], unit='nanomaggies', description = "apname_r")
    t[apname_g] = Column(aparray_g[i], unit='nanomaggies', description = "apname_g")
    t[apname_z] = Column(aparray_z[i], unit='nanomaggies', description = "apname_z")

t['shapeexp_r'] = Column(shapeexp_r, unit='arcsec', description = 'exponential shape half-light radius')
t['shapeexp_e1'] = Column(shapeexp_e1, unit='arcsec', description = 'ellipticipy argument 1')
t['shapeexp_e2'] = Column(shapeexp_e2, unit='arcsec', description = 'ellipticipy argument 2')
t['psfsize_r'] = Column(psfsize_r, unit='arcsec', description = 'weighted average PSF FWHM in r band')
t['csb_g'] = Column(csb_g, unit='mag/arcsec^2', description = 'central surface brightness in g-band')

#t['apflux_g'] = Column(apflux_g, unit='nanomaggies', description='0.75 arcsec aperture fluxes')
#t['apflux_z'] = Column(apflux_z, unit='nanomaggies', description='0.75 arcsec aperture fluxes')
#t['apflux_r'] = Column(apflux_r, unit='nanomaggies', description='0.75 arcsec aperture fluxes')

ascii.write(t, 'tractor191color.csv', format='csv', fast_writer=False, overwrite=True) 

In [15]:
a = [1, 2, 3, 4, 5, 6]
a = np.asarray(a)
mask = a > 3

cut = a < 6

mask &=cut

In [16]:
mask

array([False, False, False,  True,  True, False], dtype=bool)